In [1]:
import math
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as Data

# Data
修改channel 加入ht 长方形patch计算

In [2]:
import numpy as np
import os, sys
import random #shuffle
from netCDF4 import Dataset
def CMIPdata(Xdata, Ydata, out , myform):
    inp1 = Dataset(Xdata,'r')
    inp2 = Dataset(Ydata,'r')
    sst_1 = np.zeros((myform,140,12,24,72))
    t300_1 = np.zeros((myform,140,12,24,72))

    #首年序列
    #i= 21个模式
    for i in range(myform):
        sst_1[i,:,:,:,:] = inp1.variables['sst1'][1+141*i:141+141*i,0:12,:,:]
        t300_1[i,:,:,:,:] = inp1.variables['t300'][1+141*i:141+141*i,0:12,:,:]
    #(21,140,12,24,72)
    #首年序列

    
    #flatted扁平化                                             
    sst_2 = np.zeros((myform,1680,24,72))
    t300_2 = np.zeros((myform,1680,24,72))
    for i in range(myform):
        for j in range(140):
            sst_2[i,j*12:(j+1)*12,:,:] = sst_1[i,j,:,:,:]
            t300_2[i,j*12:(j+1)*12,:,:] = t300_1[i,j,:,:,:]
    
        #(21,1680,24,72)
        #每个模式 139*12+1 = 1680 个序列 序列长度为12

    #丢弃1个月 1862.1
    sst_2 = sst_2[:,1:,:,:]
    t300_2 = t300_2[:,1:,:,:]
    #(form,1679,24,72)


    winsize = 12
    #winnum = 1679-winsize +1 #1668
    """跳跃六个月滑窗 = 6"""
    ts=6 
    #(1679-12)/6+1 =27
    winnum=int((1679-12)/ts +1) 
    
    sst_3 = np.zeros((myform,winnum, 12, 24, 72))
    t300_3 = np.zeros((myform, winnum, 12, 24, 72))
    for i in range(myform):
        for j in range(winnum):
            #print(j)
            sst_3[i,j,:,:,:] = sst_2[i,(j*ts):(j*ts)+winsize,:,:]
            t300_3[i,j,:,:,:] = t300_2[i,(j*ts):(j*ts)+winsize,:,:]
            

    sst_4 = np.zeros((int(winnum*myform),12,24,72))
    t300_4 = np.zeros((int(winnum*myform),12,24,72))
    for i in range(myform):
        sst_4[i*winnum:(i+1)*winnum,:,:,:] = sst_3[i,:,:,:,:]
        t300_4[i*winnum:(i+1)*winnum,:,:,:] = t300_3[i,:,:,:,:]


    trX = np.zeros((int(winnum*myform),12, 2, 24, 72))
    trX[:,:,0,:,:] = sst_4
    trX[:,:,1,:,:] = t300_4
    del sst_1,sst_2,sst_3,sst_4
    del t300_1,t300_2,t300_3,t300_4

    #保存np数组
    #(5859, 12, 2, 24, 72 )
    #np.save("./CMIPdata/CMIP_trX_21_ts6_out.npy",trX) 

    #label
    #平铺
    pr_1 = np.zeros(((myform),1692))#21,1692
    for i in range(myform):
        for j in range(141):
            pr_1[i,j*12:(j+1)*12] = inp2.variables['pr'][(141*i)+j,:,0,0]
            
    #out
    pr_2 =  np.zeros((myform,winnum,out))#form,1668,1
    for i in range(myform):
        for j in range(winnum):
            pr_2[i,j,:] = pr_1[i,(j*ts):(j*ts)+out]

    trY = np.zeros((int(winnum*myform),out))#form*1668,1
    for i in range(myform):
        trY[i*winnum:(i+1)*winnum,:] = pr_2[i,:,:]

    #np.save("./CMIPdata/CMIP_trY_21_ts6_out%s.npy"%out,trY)
    trY_decoder_input = np.zeros((int(winnum*myform),out))
    trY_decoder_input[:,1:] = trY[:,:-1]

    #return trX[1:,:,:] ,trY.reshape(-1,24,1)[1:,:,:] ,trY_decoder_input
    return trX ,trY.reshape(-1,24,1) ,trY_decoder_input.reshape(-1,24,1)

def GOSDAdata(Xdata, Ydata, out):
    #test data
    inp11 = Dataset(Xdata,'r')
    inp22 = Dataset(Ydata,'r')
                                              
    sst_11 = np.zeros((33,12,24,72)) #1983-2015
    t300_11 = np.zeros((33,12,24,72))

    sst_11[:,:,:,:] = inp11.variables['sst'][3:,0:12,:,:]
    t300_11[:,:,:,:] = inp11.variables['t300'][3:,0:12,:,:]
    #(33,12,24,72)

    sst_22 = np.zeros((396,24,72))
    t300_22 = np.zeros((396,24,72))

    for i in range(33):
        sst_22[i*12:(i+1)*12,:,:] = sst_11[i,:,:,:]
        t300_22[i*12:(i+1)*12,:,:] = t300_11[i,:,:,:]
    #(396,24,72)
    #丢弃一个月，便于制作滑窗序列
    sst_22 = sst_22[1:,:,:]
    t300_22 = t300_22[1:,:,:]#(395,24,72)

    #滑窗
    winsize = 12
    winnum = 395-winsize + 1 #384
    sst_33 = np.zeros((winnum, winsize, 24, 72))#(384, 12, 24, 72)
    t300_33 = np.zeros((winnum, winsize, 24, 72))

    for i in range(winnum):
        sst_33[i,:,:,:] = sst_22[i:i+12,:,:]
        t300_33[i,:,:,:] = t300_22[i:i+12,:,:]
    #(384,12,24,72)


    #channel = 2
    #testX = np.zeros((winnum,12,24,72,2))#384
    #testX[:,:,:,:,0] = sst_33
    #testX[:,:,:,:,1] = t300_33
    #sst_33 = sst_33.swapaxes(1, 3)

    testX = np.zeros((winnum, 12, 2, 24, 72))#384
    testX[:,:,0,:,:] = sst_33
    testX[:,:,0,:,:] = t300_33
    del sst_11,sst_22,sst_33
    del t300_11,t300_22,t300_33
    #testX = testX.reshape(384,12,3456)

    #label
    inpv22 = np.zeros((408))#2017-1984 +1=34
    for i in range(34):
        inpv22[i*12:(i+1)*12] = inp22.variables['pr'][i+2,:,0,0]# +2:从1984开始
        #(408)

    testY = np.zeros((winnum,out,1))
    #out = 1
    #滑窗
    for i in range(winnum):
        testY[i,:,0] = inpv22[i:i+out]#24
    #(384,out,1)

    return testX, testY

def SODAdata(Xdata, Ydata, out):
    inp1 = Dataset(Xdata,'r')
    inp2 = Dataset(Ydata,'r')

    #time_step = 1 month                                               
    sst_1 = np.zeros((99,12,24,72))
    t300_1 = np.zeros((99,12,24,72))

    sst_1[:,:,:,:] = inp1.variables['sst'][1:,0:12,:,:]
    t300_1[:,:,:,:] = inp1.variables['t300'][1:,0:12,:,:]
    #(99,12,24,72)
    #1872-1970

    sst_2 = np.zeros((1188,24,72))
    t300_2 = np.zeros((1188,24,72))

    for i in range(99):
        sst_2[i*12:(i+1)*12,:,:] = sst_1[i,:,:,:]
        t300_2[i*12:(i+1)*12,:,:] = t300_1[i,:,:,:]
    #(1188,24,72)
    #丢弃一个月，便于制作滑窗序列
    sst_2 = sst_2[1:,:,:]
    t300_2 = t300_2[1:,:,:]#(1187,24,72)

    sst_3 = np.zeros((1176,12,24,72))
    t300_3 = np.zeros((1176,12,24,72))
    #滑窗
    for i in range(1176):
        sst_3[i:,:,:] = sst_2[i:i+12,:,:]
        t300_3[i,:,:,:] = t300_2[i:i+12,:,:]
    #(1176,12,24,72)

    #channel = 2
    #trX = np.zeros((1176,12,24,72,2))
    #trX[:,:,:,:,0] = sst_3
    #trX[:,:,:,:,1] = t300_3
    trX = np.zeros((1176,12,2,24,72))
    trX[:,:,0,:,:] = sst_3
    trX[:,:,1,:,:] = t300_3
    del sst_1,sst_2,sst_3
    del t300_1,t300_2,t300_3

    #保存np数组
    #np.save("./SODAdata/SODA_trX_ts1_out.npy",trX)

    #label
    inpv2 = np.zeros((1200))
    for i in range(100):
        inpv2[i*12:(i+1)*12] = inp2.variables['pr'][i,:,0,0]
    #(1200)

    #out = 1
    trY = np.zeros((1176,out,1))
    #滑窗
    for i in range(1176):
        trY[i,:,0] = inpv2[i:i+out]#24
    #(1176,24)
    
    
    trY_decoder_input = np.zeros((1176,out,1))
    trY_decoder_input[:,1:,:] = trY[:,:-1,:]

    #np.save("./SODAdata/SODA_trY_ts1_out%s.npy"%out,trY) 
    return trX , trY, trY_decoder_input

trX, trY, trY_decoder_input = CMIPdata('/home/d/Q/liyanqiu/saconvlstm/CMIP5.input.36mon.1861_2001.nc',\
                                          '/home/d/Q/liyanqiu/saconvlstm/CMIP5.label.12mon.1863_2003.nc', 24,  21)

testX, testY = GOSDAdata('/home/d/Q/liyanqiu/saconvlstm/GODAS.input.36mon.1980_2015.nc',\
                         '/home/d/Q/liyanqiu/saconvlstm/GODAS.label.12mon.1982_2017.nc', 24)

trX_SODA, trY_SODA ,trY_decoder_input_SODA = SODAdata('/home/d/Q/liyanqiu/saconvlstm/SODA.input.36mon.1871_1970.nc',
                              '/home/d/Q/liyanqiu/saconvlstm/SODA.label.12mon.1873_1972.nc',24)


indices = np.arange(trX.shape[0])
np.random.shuffle(indices)
trX = trX[indices]
trY = trY[indices]
trY_decoder_input = trY_decoder_input[indices]


# shuffle
indices_soda = np.arange(trX_SODA.shape[0])
np.random.shuffle(indices_soda)
trX_SODA = trX_SODA[indices_soda]
trY_SODA = trY_SODA[indices_soda]
trY_decoder_input_SODA = trY_decoder_input_SODA[indices_soda]
class MyDataSet(Data.Dataset):
    """自定义DataLoader"""
    def __init__(self, enc_inputs,  dec_outputs):
        super(MyDataSet, self).__init__()
        self.enc_inputs = enc_inputs
        #self.dec_inputs = dec_inputs
        self.dec_outputs = dec_outputs

    def __len__(self):
        return self.enc_inputs.shape[0]

    def __getitem__(self, idx):
        return self.enc_inputs[idx],  self.dec_outputs[idx]
    
batch_size = 16
trainsize = 1000 #选择cmip训练数量
trX = trX[:trainsize]
trY = trY[:trainsize]

#train split
split_index = int(trX.shape[0]*0.8)
split_index_soda = int(trX_SODA.shape[0]*0.8)

print("split_index_soda:" + str(split_index_soda))
print("split_index:" + str(split_index))


target_month = 3
loader_train = Data.DataLoader(
    MyDataSet(
        trX[:split_index,].astype(np.float32), 
        trY[:split_index, target_month-1:target_month].astype(np.float32).reshape((-1,1))), 
    batch_size, shuffle = True)
loader_valid = Data.DataLoader(
    MyDataSet(
        trX[split_index:,].astype(np.float32), 
        trY[split_index:, target_month-1:target_month].astype(np.float32).reshape((-1,1))), 
    batch_size, shuffle=False)

loader_train_sado = Data.DataLoader(
    MyDataSet(
        trX_SODA[:split_index_soda,].astype(np.float32), 
        trY_SODA[:split_index_soda, target_month-1:target_month].astype(np.float32).reshape((-1,1))), 
    batch_size, shuffle = True)
loader_valid_soda = Data.DataLoader(
    MyDataSet(
        trX_SODA[split_index_soda:,].astype(np.float32), 
        trY_SODA[split_index_soda:, target_month-1:target_month].astype(np.float32).reshape((-1,1))), 
    batch_size, shuffle=False)

#testdataloader

loader_test = Data.DataLoader(
    MyDataSet(
        testX[:,:].astype(np.float32), 
        testY[:, target_month-1:target_month].astype(np.float32).reshape((-1,1))), 
    batch_size, shuffle = False)

testY = testY[:, target_month-1:target_month].astype(np.float32).reshape((-1,1))

print(trX.shape)
print(trX_SODA .shape)
print(testX.shape)
print(testY.shape)
print(len(testX))
print(trX_SODA.shape)
print("----------------------------Train Data----------------------------")
print(trX[:split_index,].astype(np.float32).shape)
print(trY[:split_index, target_month-1:target_month].reshape((-1,1)).shape)

split_index_soda:940
split_index:800
(1000, 12, 2, 24, 72)
(1176, 12, 2, 24, 72)
(384, 12, 2, 24, 72)
(384, 1)
384
(1176, 12, 2, 24, 72)
----------------------------Train Data----------------------------
(800, 12, 2, 24, 72)
(800, 1)


# Model

In [3]:
from torch import nn, einsum
import torch.nn.functional as F
from einops import rearrange, repeat


class PreNorm(nn.Module):
    def __init__(self, dim, fn):
        super().__init__()
        self.fn = fn
        self.norm = nn.LayerNorm(dim)

    def forward(self, x, *args, **kwargs):
        x = self.norm(x)
        return self.fn(x, *args, **kwargs)
    
#feed forward
class GEGLU(nn.Module):
    def forward(self, x):
        x, gates = x.chunk(2, dim = -1) # 在给定的维度上将张量进行分块。沿着-1分成两块
        return x * F.gelu(gates)

class FeedForward(nn.Module):
    def __init__(self, dim, mult = 4, dropout = 0.):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(dim, dim * mult * 2),
            GEGLU(),
            nn.Dropout(dropout),
            nn.Linear(dim * mult, dim)#geglu大小除以2
        )

    def forward(self, x):
        return self.net(x)
    
#attention
def attn(q, k, v):
    sim = einsum('b i d, b j d -> b i j', q, k)
    attn = sim.softmax(dim = -1)
    out = einsum('b i j, b j d -> b i d', attn, v)
    return out

class Attention(nn.Module):
    def __init__(
        self,
        dim,
        dim_head = 64,
        heads = 8,
        dropout = 0.
    ):
        super().__init__()
        self.heads = heads
        self.scale = dim_head ** -0.5
        inner_dim = dim_head * heads

        self.to_qkv = nn.Linear(dim, inner_dim * 3, bias = False)
        self.to_out = nn.Sequential(
            nn.Linear(inner_dim, dim),
            nn.Dropout(dropout)
        )

    def forward(self, x, einops_from, einops_to, **einops_dims):
        h = self.heads
        #print(x.shape)[16, 28, 64]
        q, k, v = self.to_qkv(x).chunk(3, dim = -1)#参数在to_qkv里边
        #print(q.shape)[16, 28, 256]
        q, k, v = map(lambda t: rearrange(t, 'b n (h d) -> (b h) n d', h = h), (q, k, v))#qkv头参数
        q *= self.scale

        # splice out classification token at index 1
        # 在索引1处拼接出分类标记
        (cls_q, q_), (cls_k, k_), (cls_v, v_) = map(lambda t: (t[:, 0:1], t[:, 1:]), (q, k, v))
        #[64, 1, 64]
        #[64, 27, 64]
        
        """【timesformer patch 2修改部分，对classtoken的融合机制进行修改】"""

        # let classification token attend to key / values of all patches across time and space
        # 让cla token关注跨时间和空间的所有patch的键/值
        #cls_out = attn(cls_q, k, v)# ！！！对q的处理
        #[64, 1, 64]

        # rearrange across time or space
        # 将空间/时间维度移动到批处理维度以处理时间/空间注意力
        q_, k_, v_ = map(lambda t: rearrange(t, f'{einops_from} -> {einops_to}', **einops_dims), (q_, k_, v_))

        # expand cls token keys and values across time or space and concat
        # 跨越时间或空间扩展clstoken的键和值，并将其连接起来。
        r = q_.shape[0] // cls_k.shape[0]# 控制时间或者空间 n or f
        cls_q, cls_k, cls_v = map(lambda t: repeat(t, 'b () d -> (b r) () d', r = r), (cls_q, cls_k, cls_v))
        #print(cls_q.shape)
        
        
        q_ = torch.cat((cls_q, q_), dim = 1)
        k_ = torch.cat((cls_k, k_), dim = 1)
        v_ = torch.cat((cls_v, v_), dim = 1)
        #[576, 3+1, 64] [192, 9+1, 64]


        # attention
        #q_ [576, 3, 64] 没有class token
        out = attn(q_, k_, v_)
        # out [576, 3, 64] /[192, 9, 64]
        
        
        cls_out, out = out[:, 0:1], out[:, 1:]
        # merge back time or space 合并时间或空间
        out = rearrange(out, f'{einops_to} -> {einops_from}', **einops_dims)#恢复
        # [64, 27, 64] 没有class token
        cls_out = rearrange(cls_out, f'{einops_to} -> {einops_from}', **einops_dims)#恢复
        
        cls_out_new = torch.mean(cls_out, dim=1).reshape(cls_out.shape[0], 1, cls_out.shape[2])
        #print(cls_out_new.shape)
        

        # concat back the cls token 连接回来的cls token
        out = torch.cat((cls_out_new, out), dim = 1)#[16, , d]
        #[64, 28, 64]

        # merge back the heads 合并后的头
        out = rearrange(out, '(b h) n d -> b n (h d)', h = h)# [16, 109, inner_dim] inner_dim = dim_head * heads所有头的平铺
        #  [16, 28, 256]

        # combine heads out 合并head输出
        return self.to_out(out) # batch * dim
    
    
# main classes
class TimeSformer(nn.Module):
    def __init__(
        self,
        *,
        dim,
        num_frames,
        num_classes,
        image_size = 72,
        patch_size = 12,
        channels = 2,
        depth = 12,
        heads = 8,
        dim_head = 64,
        attn_dropout = 0.,
        ff_dropout = 0.
    ):
        super().__init__()
        #符合则往下运行
        assert image_size % patch_size == 0, 'Image dimensions must be divisible by the patch size.'

        num_patches = (image_size // patch_size) ** 2 #平方个数 //向下取整
        num_positions = num_frames * num_patches # 12*num_patches
        patch_dim = channels * patch_size * (patch_size//3)# patch大小

        self.patch_size = patch_size
        self.to_patch_embedding = nn.Linear(patch_dim, dim)
        self.pos_emb = nn.Embedding(num_positions + 1, dim)# 总共出现num_positions + 1个词,多了一个cls_token patch 用多少维来表示一个符号
        self.cls_token = nn.Parameter(torch.randn(1, dim))# [1, dim] 一个可学习的参数

        self.layers = nn.ModuleList([])
        for _ in range(depth):
            self.layers.append(nn.ModuleList([
                PreNorm(dim, Attention(dim, dim_head = dim_head, heads = heads, dropout = attn_dropout)), # Time attention
                PreNorm(dim, Attention(dim, dim_head = dim_head, heads = heads, dropout = attn_dropout)), # Spatial attention
                PreNorm(dim, FeedForward(dim, dropout = ff_dropout)) # Feed Forward
            ]))
            #append到layers中
            
        # 多尺度CNN部分
        self.projs = nn.ModuleList()
        kernel_size = [17, 9, 5]
        out_chans = 10
        for ps in kernel_size:
            stride = 1
            padding = (ps - stride) // 2
            self.projs.append(nn.Conv2d(2, out_chans, kernel_size=ps, stride=stride, padding=padding))
            
        """
        self.to_out = nn.Sequential(
            nn.LayerNorm(dim),
            nn.Linear(dim, num_classes)
        )
        """
    
        self.to_out = nn.Linear(dim, num_classes)

    def forward(self, video):
        # (batch x frames x channels x height x width)
        b, f, _, h, w, *_, device, p = *video.shape, video.device, self.patch_size
        video = rearrange(video, 'b f c h w -> (b f) c h w')
        xs = []
        for i in range(len(self.projs)):
            tx = self.projs[i](video)
            #print(i)
            #print(tx.shape)#[192, 10, 23, 71]
            tx = rearrange(tx, '(b1 f1) c h w -> b1 f1 c h w', b1 = b, f1 = f)
            xs.append(tx)
        msvideo = torch.cat(xs, dim=2) #按channel维度拼接
        
        #print(x.shape)#torch.Size([16, 12, 30, 24, 72])
        h, w= msvideo.shape[3], msvideo.shape[4]
        assert h % (p//3) == 0 and w % p == 0, f'height {h} and width {w} of video must be divisible by the patch size {p}'
        #print(h) 24
        #print(w) 72

        n = (h // (p//3)) * (w // p)# patch个数 3*3个patch p = 24
        #print(n)
        # 将w*h*c图片切分为N个p1*p2*c的图像块，序列长度变成f*h*w/(p*p/3)
        msvideo = rearrange(msvideo, 'b f c (h p1) (w p2) -> b (f h w) (p1 p2 c)', p1 = p//3, p2 = p)
        #print("patch finish")
        #print(video.shape) [16, 27, 384]
        # torch.Size([16, 108, 384]) 9*12 = 108
        
        #数据处理完毕
        tokens = self.to_patch_embedding(msvideo)# 对每个patch进行嵌入

        cls_token = repeat(self.cls_token, 'n d -> b n d', b = b)# [batch_size, 1, dim]
        x =  torch.cat((cls_token, tokens), dim = 1)#按照frames维度进行拼接 [batch_size, (f h w)+1, dim] 对每一个样本增加了一个cls_token
        x += self.pos_emb(torch.arange(x.shape[1], device = device)) #位置编码
        #print(x.shape)[16, 28, 64]

        for (time_attn, spatial_attn, ff) in self.layers:#layers 循环 [ [1, 2, 3], [1, 2, 3], [1, 2, 3] ] d二维list
            x = time_attn(x, 'b (f n) d', '(b n) f d', n = n) + x
            x = spatial_attn(x, 'b (f n) d', '(b f) n d', f = f) + x
            x = ff(x) + x
            
        # print(x.shape) [16, 109, 64]

        cls_token = x[:, 0] #把patch做一个融合 取cls_token！
        #print(cls_token.shape)
        
        return self.to_out(cls_token)
    
    
#Parameters
device = 'cuda'
DIM = 64
IMAGE_SIZE = 72
PATCH_SIZE = 18 #24 18 12 9 6
NUM_CLASSES = 1
NUM_FRAMES = 12
DEPTH = 2 # block个数
HEADS = 8 
DIM_HEAD = 64
ATTN_DROPOUT = 0.
FF_DROPOUT = 0.

# (batch x frames x channels x height x width)
model = TimeSformer(dim = DIM, image_size = IMAGE_SIZE, patch_size = PATCH_SIZE, num_frames = NUM_FRAMES, \
        num_classes = NUM_CLASSES, depth = DEPTH, heads = HEADS, dim_head = DIM_HEAD, attn_dropout = ATTN_DROPOUT, \
            ff_dropout = FF_DROPOUT).to(device)
criterion = nn.SmoothL1Loss()
#criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters())
print("if on cuda:",next(model.parameters()).is_cuda)

if on cuda: True


# Train

In [4]:
import pandas as pd 
import datetime
dfhistory = pd.DataFrame(columns = ["epoch","loss","val_loss"]) 
print("Start Training...")
nowtime = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
print("=========="*8 + "%s"%nowtime)
bestloss = float("inf")
epoch = 15
model_epoch = 0
for epoch in range(1,epoch+1):
    #train----------------------------------------------------
  model.train()
  loss_sum = 0.0
  step = 1
  for step,(enc_inputs, dec_outputs) in enumerate(loader_train, 1):
      optimizer.zero_grad()

      # 正向
      enc_inputs, dec_outputs = enc_inputs.to(device), dec_outputs.to(device)
      y_pred = model(enc_inputs)
      
      loss = criterion(y_pred, dec_outputs) 

      # 反向
      loss.backward()
      optimizer.step()

      # print
      loss_sum += loss.item()
      #if step%40 == 0:
        #print(("[step = %d] loss: %.3f, ") % (step, loss_sum/step))


  #valid
  model.eval()
  val_loss_sum = 0.0
  val_step = 1

  for val_step, (enc_inputs, dec_outputs) in enumerate(loader_valid, 1):
    with torch.no_grad():# 节点不进行求梯度
      enc_inputs, dec_outputs = enc_inputs.to(device), dec_outputs.to(device)
      outputs = model(enc_inputs)
      #val_loss = criterion(outputs.view(-1), dec_outputs.view(-1)) 
      val_loss =criterion(outputs, dec_outputs) 

    val_loss_sum += val_loss.item()


  # log
  info = (epoch, loss_sum/step, val_loss_sum/val_step)
  dfhistory.loc[epoch-1] = info # epoch从1开始 index -1

  # print
  print(("\nEPOCH = %d, loss = %.3f,"+" val_loss = %.3f, ")% info)
  nowtime = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
  print("\n"+"=========="*8 + "%s"%nowtime)

  if val_loss_sum <= bestloss: #换成小于等于比小于效果更好
        bestloss = val_loss_sum
        torch.save(model,'bestmodel.pkl')
        model_epoch = epoch
        #print("bestmodel updata")
        

print('Finished Training...')

out = []
# batch_size
for step,(inputs, _) in enumerate(loader_test, 1):
    pred = model(inputs.to(device))
    out.append(pred.squeeze().cpu().detach().numpy())

out = np.array(out)    
print(out.shape)

#——————————————cor————————————————
cor = np.zeros((1))
cor = np.corrcoef(testY[:,0],out.reshape((-1)))[0,1]
print(cor)

Start Training...
================================================================================2022-08-14 19:59:35


RuntimeError: Function MmBackward returned an invalid gradient at index 0 - got [3072, 216] but expected shape compatible with [3072, 3240]

In [45]:
testY[:,0]

array([-0.70335156, -0.9443807 , -1.1383914 , -1.158753  , -1.1583354 ,
       -1.180532  , -1.1754164 , -1.0011051 , -0.80188906, -0.6772756 ,
       -0.52563614, -0.43219197, -0.36762857, -0.33536553, -0.28663447,
       -0.24307424, -0.3515917 , -0.53687763, -0.6845002 , -0.6343052 ,
       -0.4556363 , -0.19504076,  0.00893648,  0.22462256,  0.45296106,
        0.6640037 ,  0.8641342 ,  1.0855862 ,  1.2000219 ,  1.2075744 ,
        1.0670185 ,  1.0259333 ,  1.0872009 ,  1.2204103 ,  1.3105447 ,
        1.3004302 ,  1.2650048 ,  1.2087082 ,  1.0506958 ,  0.8041709 ,
        0.4654007 ,  0.11666613, -0.30131388, -0.82851887, -1.3176688 ,
       -1.5566994 , -1.4677974 , -1.3122475 , -1.5108223 , -1.9338089 ,
       -2.2517302 , -2.160442  , -1.7987889 , -1.5098666 , -1.281021  ,
       -1.0888195 , -0.82842743, -0.5189633 , -0.33908117, -0.2899837 ,
       -0.32445404, -0.38313252, -0.33575767, -0.21028854, -0.01515023,
        0.04208653,  0.03992243,  0.04767451,  0.04120988,  0.04

In [65]:
out.reshape((-1))

array([-1.3421562e-01, -2.4632950e-01, -1.7897381e-01,  7.7676371e-02,
        1.1805372e-01,  7.6637939e-02,  1.2494746e-01,  1.4293577e-01,
        7.9432011e-02,  1.6330704e-01,  2.4310900e-01,  9.5570646e-03,
       -9.6604794e-02, -2.4112675e-01,  2.3673270e-02,  2.9296838e-03,
       -3.1318478e-03,  7.5737089e-02,  8.5582614e-02,  3.4403542e-01,
        3.7450570e-01,  2.3445462e-01,  1.8400829e-02,  1.9976622e-01,
        3.5949492e-01,  1.4563480e-01, -1.3909495e-01, -2.9757306e-01,
       -3.3816707e-01, -2.3372182e-01, -8.2806364e-02,  2.1430112e-01,
        6.6312408e-01,  8.4901357e-01,  6.3849229e-01,  5.3906929e-01,
        4.4473636e-01,  1.8007579e-01,  3.6751112e-01,  4.8600695e-01,
        3.0203724e-01, -1.0829061e-01, -3.9648983e-01, -2.1320309e-01,
       -2.1445838e-01, -2.8444424e-01, -3.8680834e-01, -5.2365291e-01,
       -2.5605634e-01, -1.2606712e-01, -1.9090630e-01, -2.4754873e-01,
       -2.3122174e-01, -4.5962575e-01, -5.2032799e-01, -5.2521116e-01,
      

# soda

In [23]:
import pandas as pd 
import datetime
dfhistory = pd.DataFrame(columns = ["epoch","loss","val_loss"]) 
print("Start Training...")
nowtime = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
print("=========="*8 + "%s"%nowtime)
bestloss = float("inf")
epoch = 5
model_epoch = 0
for epoch in range(1,epoch+1):
    #train----------------------------------------------------
  model.train()
  loss_sum = 0.0
  step = 1
  for step,(enc_inputs, dec_outputs) in enumerate(loader_train_sado, 1):
      optimizer.zero_grad()

      # 正向
      enc_inputs, dec_outputs = enc_inputs.to(device), dec_outputs.to(device)
      y_pred = model(enc_inputs)
      
      loss = criterion(y_pred, dec_outputs) 

      # 反向
      loss.backward()
      optimizer.step()

      # print
      loss_sum += loss.item()
      #if step%40 == 0:
        #print(("[step = %d] loss: %.3f, ") % (step, loss_sum/step))


  #valid
  model.eval()
  val_loss_sum = 0.0
  val_step = 1

  for val_step, (enc_inputs, dec_outputs) in enumerate(loader_valid_soda, 1):
    with torch.no_grad():# 节点不进行求梯度
      enc_inputs, dec_outputs = enc_inputs.to(device), dec_outputs.to(device)
      outputs = model(enc_inputs)
      #val_loss = criterion(outputs.view(-1), dec_outputs.view(-1)) 
      val_loss =criterion(outputs, dec_outputs) 

    val_loss_sum += val_loss.item()


  # log
  info = (epoch, loss_sum/step, val_loss_sum/val_step)
  dfhistory.loc[epoch-1] = info # epoch从1开始 index -1

  # print
  print(("\nEPOCH = %d, loss = %.3f,"+" val_loss = %.3f, ")% info)
  nowtime = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
  print("\n"+"=========="*8 + "%s"%nowtime)

  if val_loss_sum <= bestloss: #换成小于等于比小于效果更好
        bestloss = val_loss_sum
        torch.save(model,'bestmodel_soda.pkl')
        model_epoch = epoch
        #print("bestmodel updata")
        

print('Finished Training...')

Start Training...
================================================================================2022-04-01 17:48:55

EPOCH = 1, loss = 0.218, val_loss = 0.207, 

================================================================================2022-04-01 17:48:56

EPOCH = 2, loss = 0.120, val_loss = 0.090, 

================================================================================2022-04-01 17:48:56

EPOCH = 3, loss = 0.057, val_loss = 0.070, 

================================================================================2022-04-01 17:48:56

EPOCH = 4, loss = 0.034, val_loss = 0.056, 

================================================================================2022-04-01 17:48:57

EPOCH = 5, loss = 0.020, val_loss = 0.037, 

================================================================================2022-04-01 17:48:57
Finished Training...


# Evaluation

In [24]:
out = []
# batch_size
for step,(inputs, _) in enumerate(loader_test, 1):
    pred = model(inputs.to(device))
    out.append(pred.squeeze().cpu().detach().numpy())

out = np.array(out)    
print(out.shape)

#——————————————cor————————————————
cor = np.zeros((1))
cor = np.corrcoef(testY[:,0],out.reshape((-1)))[0,1]
print(cor)

(24, 16)
-0.011562084762578915


In [73]:
out = []
# batch_size
model=torch.load('bestmodel_soda.pkl')
for step,(inputs, _) in enumerate(loader_test, 1):
    pred = model(inputs.to(device))
    out.append(pred.squeeze().cpu().detach().numpy())

out = np.array(out)    
print(out.shape)

#——————————————cor————————————————
cor = np.zeros((1))
cor = np.corrcoef(testY[:,0],out.reshape((-1)))[0,1]
print(cor)

(24, 16)
0.7452393533846223


In [14]:
out = []
# batch_size
model=torch.load('bestmodel.pkl')
for step,(inputs, _) in enumerate(loader_test, 1):
    pred = model(inputs.to(device))
    out.append(pred.squeeze().cpu().detach().numpy())

out = np.array(out)    
print(out.shape)

#——————————————cor————————————————
cor = np.zeros((1))
cor = np.corrcoef(testY[:,0],out.reshape((-1)))[0,1]
print(cor)

(24, 16)
0.5400710080447104


In [17]:
print(model_epoch)

0


In [8]:
testY[:16,0]

array([-0.7828678 , -0.64715624, -0.68216705, -0.86499506, -0.8866562 ,
       -0.8267274 , -0.6234623 , -0.70335156, -0.9443807 , -1.1383914 ,
       -1.158753  , -1.1583354 , -1.180532  , -1.1754164 , -1.0011051 ,
       -0.80188906], dtype=float32)

In [9]:
out[0]

array([ 0.3945299 ,  0.21358556, -0.55354697, -0.40999395, -0.8203296 ,
       -0.46780396, -0.0918841 , -0.05698049, -0.14935665,  0.16960537,
        0.2032471 ,  0.13740295,  0.19149709,  0.08952218,  0.1406549 ,
        0.03880312], dtype=float32)

In [ ]:
pre = model(torch.from_numpy(testX[:10].astype(np.float32)).to(device))
pre.cpu().detach().numpy()
pre.shape

In [6]:
a = np.array([[0,1,2], [3,4,5]])
a.resize(3, 3)
print(a)


[[0 1 2]
 [3 4 5]
 [0 0 0]]


In [41]:
a = np.array([[0,1,2], [3,4,5]])
a.reshape((-1))

array([0, 1, 2, 3, 4, 5])